# Objective

The aim of this project is to identify whether a coupon code is redeemed or not

In [41]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train/train.csv')
item_data = pd.read_csv('train/item_data.csv')
camp_data = pd.read_csv('train/campaign_data.csv')
cust_demo = pd.read_csv('train/customer_demographics.csv')
cust_trans = pd.read_csv('train/customer_transaction_data.csv')
item_map = pd.read_csv('train/coupon_item_mapping.csv')

In [81]:
test = pd.read_csv('test/test.csv')
test['redemption_status'] = 0
test['data_type'] = 'test'
train['data_type'] = 'train'

In [87]:
df = pd.concat([train, test], ignore_index=True)

In [88]:
df.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,data_type
0,1,13,27,1053,0,train
1,2,13,116,48,0,train
2,6,9,635,205,0,train
3,7,13,644,1050,0,train
4,9,8,1017,1489,0,train


In [89]:
df.tail()

,id,campaign_id,coupon_id,customer_id,redemption_status,data_type
128590,128584,20,843,501,0,test
128591,128588,25,415,481,0,test
128592,128591,25,596,1336,0,test
128593,128593,22,518,748,0,test
128594,128594,18,988,851,0,test


In [79]:
test.head()

,id,campaign_id,coupon_id,customer_id,redemption_status
0,3,22,869,967,0
1,4,20,389,1566,0
2,5,22,981,510,0
3,8,25,1069,361,0
4,10,17,498,811,0


From the test data, it is clear that we have to create a profile for each customer based on campaign_id and coupon_id

In [4]:
train.head()

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0


In [7]:
item_data.head()

,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery
3,4,56,Local,Grocery
4,5,56,Local,Grocery


<b>Campaign data</b> can have a feature called duration. <b>Customer_Demo</b> has missing data.

In [8]:
camp_data.head()

,campaign_id,campaign_type,start_date,end_date
0,24,Y,21/10/13,20/12/13
1,25,Y,21/10/13,22/11/13
2,20,Y,07/09/13,16/11/13
3,23,Y,08/10/13,15/11/13
4,21,Y,16/09/13,18/10/13


In [9]:
cust_demo.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5
2,7,26-35,NaN,0,3,1,3
3,8,26-35,NaN,0,4,2,6
4,10,46-55,Single,0,1,NaN,5


In [10]:
item_map.head()

,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77


In [11]:
cust_trans.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


# EDA

Checking Campaign data

In [14]:
camp_data[camp_data['campaign_id']==13]

,campaign_id,campaign_type,start_date,end_date
10,13,X,19/05/13,05/07/13


In [16]:
camp_data[['campaign_id','campaign_type']].groupby(['campaign_type']).count()

,campaign_id
campaign_type,
X,6
Y,22


In [17]:
camp_data.shape

(28, 4)

Checking Item Mapping

In [23]:
item_map[['coupon_id', 'item_id']].groupby(['item_id']).count().head()

,coupon_id
item_id,
1,2
4,3
7,2
10,4
12,2


There are 36k unique items which have coupon codes. So almost 50% have coupons while the rest none

In [31]:
item_map[['coupon_id', 'item_id']].groupby(['item_id']).count().shape

(36289, 1)

Checking Item_data

In [25]:
item_data.head()

,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery
3,4,56,Local,Grocery
4,5,56,Local,Grocery


In [27]:
item_data.isnull().sum()

item_id       0
brand         0
brand_type    0
category      0
dtype: int64

In [32]:
item_data[['item_id', 'brand_type']].groupby(['brand_type']).count()

,item_id
brand_type,
Established,62842
Local,11224


In [33]:
item_data[['item_id', 'category']].groupby(['category']).count()

,item_id
category,
Alcohol,303
Bakery,1679
"Dairy, Juices & Snacks",2425
Flowers & Plants,664
Fuel,14
Garden,102
Grocery,32448
Meat,2080
Miscellaneous,385


Checking train data

In [34]:
train.isnull().sum()

id                   0
campaign_id          0
coupon_id            0
customer_id          0
redemption_status    0
dtype: int64

In [44]:
train[['id', 'redemption_status', 'campaign_id']].groupby(['campaign_id', 'redemption_status']).count().head()

id
campaign_id redemption_status     
1           0                  142
            1                    1
2           0                  763
            1                    5
3           0                  406

In [43]:
np.count_nonzero(train.campaign_id.unique())

18

Understanding cust_demo

In [45]:
cust_demo.isnull().sum()

customer_id         0
age_range           0
marital_status    329
rented              0
family_size         0
no_of_children    538
income_bracket      0
dtype: int64

In [52]:
train[['customer_id', 'id']].groupby(['customer_id']).count().head()

,id
customer_id,
1,94
3,42
4,23
5,52
6,32


50% have cust_demo data, while rest dont

In [90]:
len(set(df.customer_id.unique()).intersection(set(cust_demo.customer_id.unique())))

760

In [94]:
len(df.customer_id.unique())

1582

Checking customer transation

In [70]:
cust_trans.isnull().sum()

date               0
customer_id        0
item_id            0
quantity           0
selling_price      0
other_discount     0
coupon_discount    0
dtype: int64

In [71]:
cust_trans.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


Full customer transaction data is available

In [122]:
len(set(cust_trans.customer_id.unique()).intersection(set(df.customer_id.unique())))

1582

In [123]:
len(df.customer_id.unique())

1582

Item Mapping available for all

In [124]:
len(set(df.coupon_id.unique()).intersection(set(item_map.coupon_id.unique())))

1116

In [125]:
len(df.coupon_id.unique())

1116

In [126]:
len(set(train[train['redemption_status']==1]['customer_id'].unique()).intersection(set(train[train['redemption_status']==0]['customer_id'].unique())))

247

In [116]:
train[(train['customer_id']==1) & (train['redemption_status']==1)]

,id,campaign_id,coupon_id,customer_id,redemption_status,data_type
25380,41644,8,473,1,1,train
57575,94621,8,772,1,1,train
